<a href="https://colab.research.google.com/github/yuki020936/Option-NN/blob/main/YUKI_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pip install -r requirement.txt

In [1]:
import pdb #debug
import numpy as np
import os
from time import sleep #sleep(?)暫停?秒
import signal #處理訊號（signal），這些訊號是作業系統在特定情況下發送給程式的事件
import sys
import queue #list(q.queue)
import shutil
import psutil
from multiprocessing.connection import wait
import torch.multiprocessing as mp
import torch
from argparse import ArgumentParser
from dataset import DataProcessor
from utils import clear_files
from visulization import plot_model_avg_loss
import config
from train import TrainEngine
from tqdm import tqdm
import copy
import time

#torch.multiprocessing.set_sharing_strategy('file_system')
def single_running_worker():
    try:
        while True:
            # run model
            test_date, train_loss, train_mse, train_mape, train_ITMmse, train_ITMmape, train_OTMmse, train_OTMmape, test_loss, test_mse, test_mape, test_ITMmse, test_ITMmape, test_OTMmse, test_OTMmape = config.train_engine(*next(config.daily_gen)(0))
            print(f"date: {test_date}, train_mse: {train_mse:.3f}, train_mape: {train_mape:.3f}, test_mse: {test_mse:.3f}, test_mape: {test_mape:.3f}",flush=True)
            # record and update
            for v, k in zip([test_date, train_loss, train_mse, train_mape, train_ITMmse, train_ITMmape, train_OTMmse, train_OTMmape, test_loss, test_mse, test_mape, test_ITMmse, test_ITMmape, test_OTMmse, test_OTMmape], config.model_loss_list_dict):
                config.model_loss_list_dict[k] += [v]
            # plot_model_avg_loss(config.result_folder, config.args.model_type, config.model_loss_list_dict)
            end_time = time.time()
            elapsed_time = end_time - start_time
            hours = int(elapsed_time // 3600)
            minutes = int((elapsed_time % 3600) // 60)
            seconds = int(elapsed_time % 60)
            print(f"執行時間：{hours:02d}小時 {minutes:02d}分 {seconds:02d}秒")
    except StopIteration:
        pass  # EOF of generator, fine

def multi_running_check_in(rank):
    # rank is to allocate particular device
    # torch.distributed.init_process_group("nccl", rank=rank, world_size=torch.cuda.device_count()))
    try:
        q = mp.Queue()
        p = mp.Process(target=multi_running_job, args=(next(config.daily_gen)(rank), q, config.train_engine))
    except StopIteration:
        config.procs_info['EOF'] = True
        return # do nothing. that is, no new process will be allocated afterwards.

    # if we don't update these process info before create new child process, parent process may receive SIGCHLD signal while config.procs_info['process_fd_table'] does not include this new process. hence parent process fail to wait this child process. however, it (child process exits immediately) rarely happens.
    p.start()
    config.procs_info['loss_queues'].append(q)
    config.procs_info['process_list'].append(p)
    config.procs_info['process_fd_table'].append(p.sentinel)

def multi_running_job(daily_materials, loss_queue, train_engine):
    # register signal as we hope SIGINT signal can only be processed by parent process
    signal.signal(signal.SIGINT, signal.SIG_IGN)
    # run model
    test_date, train_loss, train_mse, train_mape, train_ITMmse, train_ITMmape, train_OTMmse, train_OTMmape, test_loss, test_mse, test_mape, test_ITMmse, test_ITMmape, test_OTMmse, test_OTMmape = train_engine(*daily_materials)
    print(f"[process {os.getpid()}] date: {test_date}, train_mse: {train_mse:.3f}, train_mape: {train_mape:.3f}, test_mse: {test_mse:.3f}, test_mape: {test_mape:.3f}",flush=True)
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)
    print(f"執行時間：{hours:02d}小時 {minutes:02d}分 {seconds:02d}秒")
    # return results
    loss_queue.put([train_engine.model.device, test_date, train_loss, train_mse, train_mape, train_ITMmse, train_ITMmape, train_OTMmse, train_OTMmape, test_loss, test_mse, test_mape, test_ITMmse, test_ITMmape, test_OTMmse, test_OTMmape])

def sigchld_handler(signum, frame):
    # adopt the exited child processes, collect results, identify avaliable devices
    # since signal handler allows signal reentrancy, it may require atomic operation. but it's not safe to use lock in signal handler, refer to https://stackoverflow.com/questions/12445618/accessing-shared-data-from-a-signal-handler
    # hence here ignore SIGCHLD signal before finishing cleaning up the process. note that new process should not be created before leaving sigchld_handler
    # wait() will be always valid after child process status becomes exited unless pop out the process in config.procs_info['process_fd_table']
    signal.signal(signal.SIGCHLD, signal.SIG_IGN)
    while True:
        p_sentns = wait(config.procs_info['process_fd_table'], 0)
        if len(p_sentns) == 0:
            break
        # for each exited child process
        for p_sentn in p_sentns:
            # collect results
            idx = config.procs_info['process_fd_table'].index(p_sentn)
            # to prevent if one of the child process unexpectedly is killed and hence no results can be push into the queue, the parent process will be blocked in this line
            try:
                rank, test_date, *losses = config.procs_info['loss_queues'][idx].get(block=False)  # "hang" & Remove and return an item from the queue. If optional args block is True (the default) and timeout is None (the default), "block" if necessary until an item is available.
            except queue.Empty:
                print("at least one of the child process crashed, exit!")
                os.kill(os.getpid(), signal.SIGINT)
            for v, k in zip([test_date]+losses, config.model_loss_list_dict):
                config.model_loss_list_dict[k] += [v]

            # update process info
            config.procs_info['process_list'].pop(idx)
            config.procs_info['loss_queues'].pop(idx)
            config.procs_info['process_fd_table'].pop(idx)

            # update progress bar and print results
            plot_model_avg_loss(config.result_folder, config.args.model_type, {
                k: [l for _, l in sorted(zip(config.model_loss_list_dict['date_list'], config.model_loss_list_dict[k]))]
                for k in config.model_loss_list_dict.keys()})

            if config.procs_info['fin_count'] % config.args.n_parallel_process == (config.args.n_parallel_process - 1) and config.procs_info['fin_count'] != 0:
                config.progress_bar.update()
            config.procs_info['fin_count'] = config.procs_info['fin_count'] + 1

            # identify avaliable deiveces to allocate new process onto the rank which the exited process has stayed
            config.procs_info['avaliable_device'].append(rank)
    signal.signal(signal.SIGCHLD, sigchld_handler) # remember to reset the handler again, since we don't want to actually ignore SIGCHLD signal afterwards

def sigint_handler(signum, frame):
    # ctrl+c by user
    # clear results in queues and terminate all the child process immediately
    signal.signal(signal.SIGCHLD, signal.SIG_IGN)
    print("catch KeyboardInterrupt, terminate all the child processes ...")
    for p, q in zip(config.procs_info['process_list'], config.procs_info['loss_queues']):
        try:
            _ = q.get(
                block=False)  # Warning If a process is killed using Process.terminate() or os.kill() while it is trying to use a Queue, then the data in the queue is likely to become corrupted. This may cause any other process to get an exception when it tries to use the queue later on.
        except queue.Empty:
            pass  # fine
        finally:
            p.terminate()  # send SIGTERM signal to the child process
            p.join()  # "hang" and adopt the child process
            print(f"child process {p.pid} terminated.")
    sys.exit(0)
    # original_sigint_handler = signal.getsignal(signal.SIGINT)
    # original_sigint_handler()

def multi_running_worker(test_date_list):
    # TODO: run on multiple GPUs: check the bugs related to torch.distributed.init_process_group & torch.nn.parallel.DistributedDataParallel
    config.progress_bar = tqdm(test_date_list[::config.args.n_parallel_process])
    config.progress_bar.reset()
    signal.signal(signal.SIGCHLD, sigchld_handler)
    signal.signal(signal.SIGINT, sigint_handler)

    # initialize
    for i in range(config.args.n_parallel_process):
        multi_running_check_in(rank=i % torch.cuda.device_count() if torch.cuda.is_available() and config.args.on_gpu else -1)

    while True:
        if config.procs_info['EOF'] and len(config.procs_info['process_list']) == 0: # both conditions should be met
            config.progress_bar.update()
            break
        elif len(config.procs_info['process_list']) < config.args.n_parallel_process and len(config.procs_info['avaliable_device']) > 0:
            multi_running_check_in(rank=config.procs_info['avaliable_device'].pop(0))
        else:
            # suspend until receiving a signal when there still exist at least one process
            signal.pause()
    config.progress_bar.close()

def main(start_time):
    # ----------Hyper Parameters----------#
    parser = ArgumentParser()
    parser.add_argument('--result_folder_tag', type=str, default='test')
    parser.add_argument('--on_gpu', action='store_true')
    parser.add_argument('--resume', action='store_true')
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--use_step_lr', action='store_true') # 調scheduler用，沒開是cyclic，打開是ms step
    parser.add_argument('--num_workers', type=int, default=0)
    # parser.add_argument('--window_size', type=int, default=5)
    # 若window size為1時，在執行程式抓training set會讓rolling train day start跟end在同一天
    parser.add_argument('--window_size', type=int, default=5)
    parser.add_argument('--raw_dataset_path', type=str, default='.')
    # parser.add_argument('--seed_list', type=int, nargs='+', default=[0, 1])
    parser.add_argument('--seed_list', type=int, nargs='+', default=[0, 1])
    # parser.add_argument('--sample_start_date', type=str, default='19960101')
    parser.add_argument('--sample_start_date', type=str, default='20200316')
    # parser.add_argument('--sample_times', type=int, default=36)
    parser.add_argument('--sample_times', type=int, default=1)
    # parser.add_argument('--sample_interval', type=int, default=10)
    parser.add_argument('--sample_interval', type=int, default=1)
    parser.add_argument('--n_parallel_process', type=int, default=1)
    parser.add_argument('--model_type', type=str, default="multi", choices=['multi', 'single'])
    parser.add_argument('--daily_plot_off', action='store_true')
    parser.add_argument('--residual_on', action='store_true')
    parser.add_argument('--converge_delta', type=float, default=1e-5)
    parser.add_argument('--converge_patience', type=int, default=300)
    parser.add_argument('--master_addr', type=str, default='localhost') #主機的 IP
    parser.add_argument('--master_port', type=str, default='16235') #主控機器開了一扇門（port）
    # formula of rank
    config.args = parser.parse_args(['--on_gpu', '--residual_on', '--daily_plot_off','--sample_start_date','19960101','--num_workers','4'])
    #config.args = parser.parse_args()

    # config.args.residual_on = True
    # config.args.use_step_lr = True
    print(f'on_gpu:{config.args.on_gpu},CV:{config.args.residual_on},start date:{config.args.sample_start_date},root:{config.args.raw_dataset_path},num_workers:{config.args.num_workers},n_parallel_process:{config.args.n_parallel_process}')
    config.result_folder = f"../{config.args.sample_start_date}_{config.args.batch_size}batchsize"
    if config.args.use_step_lr:
        scheduler_setting = config.ms_step_scheduler_setting
        config.result_folder += f"_{'_'.join(str(lr) for lr in scheduler_setting['init_lr_list'])}initLR_{'_'.join(str(w) for w in scheduler_setting['epoch_warmup_list'])}warmup_{scheduler_setting['epoch_decay']}decaylen_{scheduler_setting['decay_times']}decaytimes_{scheduler_setting['epoch_last']}epochlast"
    else: # cyclical lr
        scheduler_setting = config.cyclic_shceduler_setting
        config.result_folder += f"_autoLR_autoCyclen_{scheduler_setting['cur_epoch_size']}epochsize_{scheduler_setting['lr_max_to_base_ratio']}MtoBratio"
    config.result_folder += f"_{scheduler_setting['gamma']}gamma_{'_'.join(str(s) for s in config.args.seed_list)}seed"
    config.result_folder += '_residual' if config.args.residual_on else '_multi'
    config.result_folder += f'_{config.args.result_folder_tag}_test'

    clear_files(config.args.resume, config.args.sample_start_date, config.args.model_type, config.result_folder)

    print("To reproduce AAAI Gated-neural-networks-for-option-pricing paper results")
    print(config.result_folder)

    # ----------Initialization---------- #

    mp.set_start_method('spawn', force=True)
    os.environ['MASTER_ADDR'] = config.args.master_addr
    os.environ['MASTER_PORT'] = config.args.master_port
    data_processor = DataProcessor(root_path=config.args.raw_dataset_path, model_type='multi', sample_start_date=config.args.sample_start_date, sample_times=config.args.sample_times, sample_interval=config.args.sample_interval, on_gpu=config.args.on_gpu)
    test_date_list, config.daily_gen, IAO_threshold = data_processor(config.args.window_size, config.args.batch_size, config.args.num_workers, config.args.n_parallel_process, config.args.residual_on, config.args.seed_list)
    config.train_engine = TrainEngine(config.args.daily_plot_off, config.args.residual_on, config.args.use_step_lr, scheduler_setting, config.args.converge_delta, config.args.converge_patience, config.args.seed_list, IAO_threshold, config.result_folder,start_time)

    # ----------For each testing date---------- #
    if config.args.n_parallel_process == 1:
        single_running_worker()

    else:
        multi_running_worker(test_date_list)

if __name__ == '__main__':
    start_time = time.time()
    main(start_time)


on_gpu:True,CV:True,start date:19960101,root:.,num_workers:4,n_parallel_process:1
To reproduce AAAI Gated-neural-networks-for-option-pricing paper results
../19960101_32batchsize_autoLR_autoCyclen_1500epochsize_0.0625MtoBratio_0.99994gamma_0_1seed_residual_test_test
True
True
epoch 100, train_loss: 252.780, train_mse: 8996418.786, train_mape: 37.776, test_loss:2393.175, test_mse: 12358508.273, test_mape: 46.612
epoch 200, train_loss: 261.634, train_mse: 11346283.133, train_mape: 38.910, test_loss:1290.032, test_mse: 16374424.177, test_mape: 52.918


KeyboardInterrupt: 

In [4]:
import importlib
import train
import visulization
importlib.reload(train)
importlib.reload(visulization)


<module 'visulization' from '/content/visulization.py'>

In [5]:
import os
print(f'cpu核心數:{os.cpu_count()}')

import torch
import os
import psutil

def suggest_optimal_workers():
    gpu_count = torch.cuda.device_count()
    cpu_count = os.cpu_count()
    total_ram_gb = psutil.virtual_memory().total / (1024 ** 3)

    # 推估 num_workers：根據 CPU 和 RAM
    if total_ram_gb >= 16 and cpu_count >= 4:
        num_workers = min(4, cpu_count // 2)
    elif total_ram_gb >= 8:
        num_workers = min(2, cpu_count // 2)
    else:
        num_workers = 0

    # 推估 n_parallel_process：根據 GPU 數量
    if gpu_count >= 1:
        n_parallel_process = 1  # Colab A100 為單卡，建議保守設定為 1
    else:
        n_parallel_process = min(2, cpu_count // 2)  # 無 GPU 時可開多進程（風險較高）

    return {
        "gpu_count": gpu_count,
        "cpu_count": cpu_count,
        "total_ram_gb": round(total_ram_gb, 2),
        "suggested_num_workers": num_workers,
        "suggested_n_parallel_process": n_parallel_process
    }

# 執行並印出建議
suggestion = suggest_optimal_workers()
for k, v in suggestion.items():
    print(f"{k}: {v}")



cpu核心數:12
gpu_count: 1
cpu_count: 12
total_ram_gb: 83.48
suggested_num_workers: 4
suggested_n_parallel_process: 1
